# Lab 5: Hyperparameter tuning, transfer learning, and fine-tuning

### Group#: __8__  

|Name | Student ID | 
|:-:|:-:|
|Rohit Rohit | 0773987|
|Rakesh Singh | 0775942|
|Praveen Thudianplackal Jose | 0779521|
|Reshma Thomas | 0779853|

||Contribution|
|:-:|:-:|
|Rohit Rohit | Part 2 & 3  |
|Rakesh Singh | Part 4 |
|Praveen Thudianplackal Jose| Part 2 & 3 |
|Reshma Thomas | Part 4 |


The focus of this lab is on improving the results of a model. In particular, we are interested in exploring how using a pre-trained model can help improve results when we do not have a lot of data. 

#### Grading 
Individual grades will be assigned for this lab as *Part 1* is not a group activity.

For *Parts 2-4*, marks will be deducted for any extraneous code. 

#### What to submit
One zipped file (**NOT .rar**) containing:
- A copy of this notebook with:
    - Error-free code in Python/Keras
    - All code cells executed and output visible
- Images of each group members Coursera certificate
    - An image (jpeg or png) in the same directory as your notebook can be embedded using the following code in a markdown cell:
        - `<img src="my_image.jpg" width=600 align="center">`

<img src="rohit.jpg" width=600 align="center">

<img src="rakesh.jpeg" width=600 align="center">

<img src="praveen.jpeg" width=600 align="center">

<img src="reshma.jpg" width=600 align="center">

## Part 1: Learning Keras Tuner (25 marks)

Each group member should successfully complete the Coursera course [Hyperparameter Tuning with Keras Tuner](https://www.coursera.org/projects/keras-tuner) and embed an image of their course completion certificate in the cell below. 

**This is a free course that only requires registration (also free) with Coursera.**

**NOTE:** you may need to use `import keras_tuner` instead of `import kerastuner` as noted in the course files

<img src="my_image.jpg" width=600 align="center">

## Part 2: Hyperparameter tuning (25 marks)

Apply what you have learned in **Part 1** to tune a **convolutional neural network** that you create for the **Fashion MNIST** data set. Make sure: 
 - your search includes at least 4 hyperparameters
 - that you print out the results of your search 
 - that you print a summary of the best model, and 
 - that you quote the test accuracy of the best model

In [50]:
from keras.models import Model 
from keras.layers import Input 
from keras.layers import Dense 
from keras.layers import Flatten 
from keras.layers import Rescaling 
from keras.layers.convolutional import Conv2D 
from keras.layers.pooling import AveragePooling2D 
import tensorflow as tf
import keras_tuner
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Flatten, Dense
from tensorflow.keras.layers import Dense, MaxPooling2D, BatchNormalization,Dropout



(x_train, y_train), (x_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()

In [51]:
def create_model(hp):
  num_hidden_layers = 1
  num_units = 8
  dropout_rate = 0.1
  learning_rate = 0.01

  if hp:
    num_hidden_layers = hp.Choice('num_hidden_layers',values=[1,2,3])
    num_units = hp.Choice('num_units',values=[8,16,32])
    dropout_rate = hp.Float('dropout_rate', min_value=0.1,max_value=0.5)
    learning_rate = hp.Float('learning_rate',min_value=0.0001,max_value=0.01)

  model = tf.keras.models.Sequential()
  model.add (tf.keras.layers.Flatten(input_shape=(28, 28)))
  model.add(tf.keras.layers.Lambda(lambda x: x/255.))
  for _ in range(0, num_hidden_layers):
    model.add(tf.keras.layers.Dense(num_units, activation= 'relu'))
    model.add (tf.keras.layers.Dropout (dropout_rate))
  model.add(tf. keras.layers.Dense(10, activation = 'softmax'))

  model.compile(
    loss= 'sparse_categorical_crossentropy',
    optimizer=tf.keras.optimizers.Adam(learning_rate = learning_rate),
    metrics=['accuracy']
    )
  return model

In [52]:
create_model(None).summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 784)               0         
                                                                 
 lambda (Lambda)             (None, 784)               0         
                                                                 
 dense_4 (Dense)             (None, 8)                 6280      
                                                                 
 dropout_4 (Dropout)         (None, 8)                 0         
                                                                 
 dense_5 (Dense)             (None, 10)                90        
                                                                 
Total params: 6,370
Trainable params: 6,370
Non-trainable params: 0
_________________________________________________________________


In [53]:
class CustomTuner(keras_tuner.tuners.BayesianOptimization):
  def run_trail(self, trail, *args, **kwargs):
    kwargs['batch_size'] = trail.hyperparameters.Int('batch_size', 32,128, step=32)
    super(CustomTuner, self).run_trial(trail, *args, **kwargs)

In [54]:
tuner = CustomTuner(
    create_model,
    objective='val_accuracy',
    max_trials=20,
    directory='logs',
    project_name='fashion_mnist',
    overwrite=True
)

In [55]:
tuner.search_space_summary()

Search space summary
Default search space size: 4
num_hidden_layers (Choice)
{'default': 1, 'conditions': [], 'values': [1, 2, 3], 'ordered': True}
num_units (Choice)
{'default': 8, 'conditions': [], 'values': [8, 16, 32], 'ordered': True}
dropout_rate (Float)
{'default': 0.1, 'conditions': [], 'min_value': 0.1, 'max_value': 0.5, 'step': None, 'sampling': None}
learning_rate (Float)
{'default': 0.0001, 'conditions': [], 'min_value': 0.0001, 'max_value': 0.01, 'step': None, 'sampling': None}


In [56]:
tuner.search(
    x_train,y_train,
    validation_data=(x_test,y_test),
    epochs=5, verbose=False
)

INFO:tensorflow:Oracle triggered exit


In [57]:
tuner.results_summary(1)

Results summary
Results in logs/fashion_mnist
Showing 1 best trials
Trial summary
Hyperparameters:
num_hidden_layers: 1
num_units: 32
dropout_rate: 0.1
learning_rate: 0.002188523790014475
Score: 0.8629999756813049


In [58]:
model = tuner.get_best_models(num_models=1)[0]
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 784)               0         
                                                                 
 lambda (Lambda)             (None, 784)               0         
                                                                 
 dense (Dense)               (None, 32)                25120     
                                                                 
 dropout (Dropout)           (None, 32)                0         
                                                                 
 dense_1 (Dense)             (None, 10)                330       
                                                                 
Total params: 25,450
Trainable params: 25,450
Non-trainable params: 0
_________________________________________________________________


In [ ]:
_ = model.fit(
    x_train,y_train,
    validation_data=(x_test,y_test),
    epochs = 20, batch_size=128,
    callbacks = [tf.keras.callbacks.EarlyStopping(monitor='val_accuracy',patience = 3)]
    )

## Part 3: Transfer learning (25 marks)

For this part, you can choose any pre-trained network available in [keras.applications](https://keras.io/api/applications/), **except VGG16 or VGG19**. Be sure to verify that the chosen network can be used for classification. Following what we did in class, apply this model, with data augmentation, to the data you created for **Lab 4**. Compare the accuracy on test data to what you achieved in Lab 4. 

In [32]:
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import  Conv2D,MaxPooling2D,BatchNormalization,Dense,Flatten,Dropout
from tensorflow.keras.models import  Sequential
from tensorflow.keras.optimizers import Adam,RMSprop
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras.losses import  SparseCategoricalCrossentropy

In [33]:
from google.colab import drive
drive.mount("/content/drive")

from pathlib import Path
path = Path().cwd()
q = path/'drive'/'MyDrive'/'Lab_4'/'images'
print(q)

import pandas as pd
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive
/content/drive/MyDrive/Lab_4/images
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [34]:
data = q/'train/Horse'
train_df = ImageDataGenerator(rescale=1./255)
test_df = ImageDataGenerator(rescale=1./255)
validation_df = ImageDataGenerator(rescale=1./255)

In [35]:
augment_data= ImageDataGenerator(samplewise_center=True,
                                 featurewise_std_normalization=False,
                                 samplewise_std_normalization=False,
                                 zca_whitening=False,
                                 rotation_range=60,
                                 width_shift_range=0.3,
                                 height_shift_range=0.3,
                                 brightness_range=None,
                                 shear_range=0.0,
                                 zoom_range=0.0,
                                 channel_shift_range=0.0,
                                 fill_mode='nearest',
                                 cval=0.0,
                                 horizontal_flip=False,
                                 vertical_flip=False,
                                 rescale=1/255.,
                                 preprocessing_function=None,
                                 data_format=None)

In [36]:
tr_aug = train_df.flow_from_directory(q/'train',
        (148, 148), # All the images are rescaled to 148x148 to have a uniformity among all the images.
        batch_size=32, #This will pass batch of 32 at a time as passing all the images at a time will make the process very slow.
        class_mode ='binary' # Class mode is kept as binary as our output will be either yes or no. 
        )

val_aug = validation_df.flow_from_directory(q/'validation',
        (148, 148),
        batch_size=32,
        class_mode='binary'
        )

te_aug = test_df.flow_from_directory(q/'test',
        (148, 148),
        batch_size=32,
        class_mode='binary'
        )

Found 710 images belonging to 2 classes.
Found 100 images belonging to 2 classes.
Found 200 images belonging to 2 classes.


In [37]:
tr_aug_data = augment_data.flow_from_directory(q/'train',
        (148, 148),
        batch_size=32,
        class_mode ='binary'
        )

Found 710 images belonging to 2 classes.


In [38]:
import tensorflow as tf
resnet_model = Sequential()

model = tf.keras.applications.ResNet50(include_top = False,
                                       input_shape = (148,148,3),
                                       pooling = 'max', classes = 2,
                                       weights = 'imagenet')

for layers in model.layers:
  layers.trainable=False

resnet_model.add(model)
resnet_model.add(BatchNormalization())
resnet_model.add(Dropout(0.2))
resnet_model.add(Dense(64, activation='relu'))
resnet_model.add(Flatten())
resnet_model.add(Dense(128, activation = 'relu'))
resnet_model.add(Dense(2, activation = 'softmax'))

94781440/94765736 [==============================] - 1s 0us/step


In [39]:
resnet_model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 2048)              23587712  
                                                                 
 batch_normalization (BatchN  (None, 2048)             8192      
 ormalization)                                                   
                                                                 
 dropout_1 (Dropout)         (None, 2048)              0         
                                                                 
 dense_2 (Dense)             (None, 64)                131136    
                                                                 
 flatten_1 (Flatten)         (None, 64)                0         
                                                                 
 dense_3 (Dense)             (None, 128)               8320      
                                                      

In [40]:
resnet_model.compile(optimizer='adam', 
              loss=SparseCategoricalCrossentropy(from_logits=False),
              metrics=['accuracy'] )

resnet_model.fit(tr_aug_data, epochs=20, validation_data=val_aug)

Epoch 1/20
23/23 [==============================] - 102s 4s/step - loss: 0.7421 - accuracy: 0.5169 - val_loss: 0.9466 - val_accuracy: 0.5000
Epoch 2/20
23/23 [==============================] - 8s 337ms/step - loss: 0.7088 - accuracy: 0.5901 - val_loss: 1.1349 - val_accuracy: 0.5000
Epoch 3/20
23/23 [==============================] - 8s 360ms/step - loss: 0.7052 - accuracy: 0.5831 - val_loss: 0.7456 - val_accuracy: 0.4800
Epoch 4/20
23/23 [==============================] - 8s 343ms/step - loss: 0.6623 - accuracy: 0.6099 - val_loss: 0.7094 - val_accuracy: 0.4600
Epoch 5/20
23/23 [==============================] - 8s 344ms/step - loss: 0.6720 - accuracy: 0.6141 - val_loss: 0.6981 - val_accuracy: 0.5700
Epoch 6/20
23/23 [==============================] - 8s 343ms/step - loss: 0.7143 - accuracy: 0.5845 - val_loss: 0.7789 - val_accuracy: 0.4800
Epoch 7/20
23/23 [==============================] - 8s 346ms/step - loss: 0.6911 - accuracy: 0.5704 - val_loss: 0.7019 - val_accuracy: 0.5400
Epoch 8

In [41]:
aug_test_acc = resnet_model.evaluate(te_aug)
print('test_acc:', aug_test_acc)

7/7 [==============================] - 32s 5s/step - loss: 0.6468 - accuracy: 0.6000
test_acc: [0.6467734575271606, 0.6000000238418579]


## Part 4: Fine-tuning (25 marks)

Following what we did in class, fine-tune the model that you trained in **Part 3**. Compare the accuracy on test data to what you achieved in Part 3. 

In [42]:
# Creating custom model for fine-tuning

def custom_model1(hp):
 # from tensorflow.keras.applications import EfficientNetB0 
  import tensorflow as tf

  num_hidden_layers = 5
  num_units = 12
  dropout_rate = 0.3
  learning_rate = 0.001
  activation = 'relu'

  if hp: # hyperparameter space
    num_hidden_layers = hp.Choice('num_hidden_layers',values = [2,3,4])
    num_units = hp.Choice('num_untis',values = [128,64])
    dropout_rate = hp.Float('dropout_rate',min_value = 0.1,max_value = 0.3,step=0.1)
    learning_rate = hp.Float('learning_rate',min_value=0.0001,max_value=0.01)


  base_model = tf.keras.applications.ResNet50()
  inputs = base_model.input
  x = base_model.layers[-1].output
  
  for _ in range(0,num_hidden_layers):
    x = Sequential()(x)
    x = BatchNormalization()(x)
    x = Dropout(dropout_rate)(x)
    x = Dense(num_units,activation=activation)(x)

  inputs = tf.keras.Input(shape=(224, 224, 3))
  x = BatchNormalization()(inputs)
  x = base_model(x, training=False)
  x = tf.keras.layers.Dropout(0.3)(x)
  outputs = Dense(1,activation = 'sigmoid')(x)
  model = tf.keras.Model(inputs, outputs)
  

  # compiling the model
  model.compile(
    loss ='binary_crossentropy',
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate),
    metrics = ['accuracy']
    )
  
  return model

In [43]:
custom_model1(None).summary()

102981632/102967424 [==============================] - 1s 0us/step
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 batch_normalization_6 (Batc  (None, 224, 224, 3)      12        
 hNormalization)                                                 
                                                                 
 resnet50 (Functional)       (None, 1000)              25636712  
                                                                 
 dropout_7 (Dropout)         (None, 1000)              0         
                                                                 
 dense_10 (Dense)            (None, 1)                 1001      
                                                                 
Total params: 25,637,725
Trainable params: 25,584,599
Non-tr

In [44]:
# Defining custom tuner
class CustomTuner(keras_tuner.tuners.BayesianOptimization):

  def run_trail(self,trial,*args,**kwargs):
    kwargs['batch_size'] = trial.hyperparameter.Int('batch_size',32,128,step = 32)
    super(CustomTuner,self).run_trail(trial,*args,**kwargs)

In [45]:
# creating the tuner object
tuner = CustomTuner(
    custom_model1,
    objective='val_accuracy',
    max_trials = 5,
    directory='logs',
    project_name = 'resnet50',
    overwrite=True
)

In [46]:
tuner.search_space_summary()

Search space summary
Default search space size: 4
num_hidden_layers (Choice)
{'default': 2, 'conditions': [], 'values': [2, 3, 4], 'ordered': True}
num_untis (Choice)
{'default': 128, 'conditions': [], 'values': [128, 64], 'ordered': True}
dropout_rate (Float)
{'default': 0.1, 'conditions': [], 'min_value': 0.1, 'max_value': 0.3, 'step': 0.1, 'sampling': None}
learning_rate (Float)
{'default': 0.0001, 'conditions': [], 'min_value': 0.0001, 'max_value': 0.01, 'step': None, 'sampling': None}


In [47]:
tuner.search(
    tr_aug,
    validation_data = (val_aug),
    epochs = 5,verbose = True
)

Trial 5 Complete [00h 01m 02s]
val_accuracy: 0.5

Best val_accuracy So Far: 0.5
Total elapsed time: 00h 05m 13s
INFO:tensorflow:Oracle triggered exit


In [48]:
tuner.results_summary(1) 

Results summary
Results in logs/resnet50
Showing 1 best trials
Trial summary
Hyperparameters:
num_hidden_layers: 3
num_untis: 128
dropout_rate: 0.30000000000000004
learning_rate: 0.009325743984131874
Score: 0.5


In [49]:
# the best model
model = tuner.get_best_models(num_models = 1)[0] 
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 batch_normalization_3 (Batc  (None, 224, 224, 3)      12        
 hNormalization)                                                 
                                                                 
 resnet50 (Functional)       (None, 1000)              25636712  
                                                                 
 dropout_3 (Dropout)         (None, 1000)              0         
                                                                 
 dense_3 (Dense)             (None, 1)                 1001      
                                                                 
Total params: 25,637,725
Trainable params: 25,584,599
Non-trainable params: 53,126
____________________________________________